In [ ]:
import cv2
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
import tensorflow.keras.backend as k
import os
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Conv3D,UpSampling2D,ReLU,Dropout
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm.notebook import tqdm
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.layers import Layer

In [ ]:
from model_architecture_deepmag_ import Encoder,Manipulator,Decoder,Residual

In [ ]:
def build_model():
  frameA  = Input(shape=(384,384,3))
  frameB = Input(shape=(384,384,3))
  amp_factor = Input(shape = (1,1,1))

  encoder = Encoder()

  texture_A , shape_A  = encoder(frameA)
  texture_B , shape_B = encoder(frameB)

  manipulator = Manipulator()
  manipulator_output = manipulator([shape_A,shape_B,amp_factor])

  decoder = Decoder()

  output = decoder([texture_B,manipulator_output])
  model = Model(inputs = [frameA,frameB,amp_factor], outputs = output)
  
  return model

In [ ]:
model = build_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 384, 384, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 384, 384, 3  0           []                               
                                )]                                                                
                                                                                                  
 encoder_1 (Encoder)            ((None, 96, 96, 32)  156768      ['input_4[0][0]',                
                                , (None, 192, 192,                'input_5[0][0]']          

In [ ]:
model.compile(tf.keras.optimizers.Adam(0.0001), tf.keras.losses.MeanSquaredError())

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

train_dir_A = '/content/gdrive/MyDrive/extracted/frameA/10K/'
train_dir_B = '/content/gdrive/MyDrive/extracted/frameC/10K/'
y_dir = '/content/gdrive/MyDrive/extracted/amplified/10K/'

valid_dir_A= '/content/gdrive/MyDrive/extracted/frameA/100K/'
valid_dir_B= '/content/gdrive/MyDrive/extracted/frameC/100K/'
valid_dir_y = '/content/gdrive/MyDrive/extracted/amplified/100K/'

In [ ]:

A = tf.keras.preprocessing.image_dataset_from_directory(train_dir_A,labels=None,shuffle=False,image_size=(384,384),batch_size=10)

Found 10000 files belonging to 1 classes.


In [ ]:
B=tf.keras.preprocessing.image_dataset_from_directory(train_dir_B,labels=None,shuffle=False,image_size=(384,384),batch_size=10)

Found 10000 files belonging to 1 classes.


In [ ]:
y=tf.keras.preprocessing.image_dataset_from_directory(y_dir,labels=None,shuffle=False,image_size=(384,384),batch_size=10)

Found 10000 files belonging to 1 classes.


In [ ]:
valid_A = tf.keras.preprocessing.image_dataset_from_directory(valid_dir_A,labels=None,shuffle=False,image_size=(384,384),batch_size=10)

Found 10000 files belonging to 1 classes.


In [ ]:
valid_B =tf.keras.preprocessing.image_dataset_from_directory(valid_dir_B,labels=None,shuffle=False,image_size=(384,384),batch_size=10)

Found 10000 files belonging to 1 classes.


In [ ]:
valid_y =tf.keras.preprocessing.image_dataset_from_directory(valid_dir_y,labels=None,shuffle=False,image_size=(384,384),batch_size=10)

Found 10000 files belonging to 1 classes.


In [ ]:
mf = pd.read_csv('train_mf.txt',names=['mf'])

In [ ]:
mf_li=[]
for i in mf['mf']:
  mf_li.append(tf.Variable([[[i]]]))

In [ ]:
# obtaining magnification factor of 10000 images we are going to train 
# the model on. the values are stacked together to form a batchsize of 10
mf_li_10k=[]
a =0
b= 10
for i in range(1000):
  if b<10000:
    val = tf.stack(mf_li[a:b],axis=0)
    a+=10
    b+=10
  else:
    val = tf.stack(mf_li[a:10000],axis=0)
  mf_li_10k.append(val)


In [ ]:
# obtaining magnification factor of 10000 images we are going to use as validation data. the values are stacked together to form a batchsize of 10
# validation data is data from 100k folder
mf_li_valid =[]
a =90000
b= 90010
for i in range(1000):
  if b<100000:
    val = tf.stack(mf_li[a:b],axis=0)
    a+=10
    b+=10
  else:
    val = tf.stack(mf_li[a:100000],axis=0)
  mf_li_valid.append(val)


In [ ]:
# create iterator  from the dataset so that 10 images can be fed at a time to the model
iteratorA =iter(A)
iteratorB =iter(B)
iteratory =iter(y)
iterator_val_A= iter(valid_A)
iterator_val_B = iter(valid_B)
iterator_val_y = iter(valid_y)

In [ ]:
new_model = load_model('/content/gdrive/MyDrive/project deepmag/saved model 70k/')

In [ ]:
new_model.compile(tf.keras.optimizers.Adam(0.00001), tf.keras.losses.MeanSquaredError())

In [ ]:
loss_training=[]
loss_val=[]
def custom_train_loop(iterations):
  for i in range(iterations):
    # generate next batch of data from iterator
      a = iteratorA.get_next()
      b = iteratorB.get_next()
      y_= iteratory.get_next()
      val_a = iterator_val_A.get_next()
      val_b = iterator_val_B.get_next()
      val_y = iterator_val_y.get_next()
      # obtain corresponding mf values from the list
      mf = mf_li_10k[i]
      mf_y = mf_li_valid[i]
      new_model.fit(x=[a,b,mf],y=y_,verbose=2,validation_data=([val_a,val_b,mf_y],val_y))
      loss_training.append(new_model.history.history['loss'])
      loss_val.append(new_model.history.history['val_loss'])
      if i%10 ==0:
        # saving the model after processing a batch of 100 images(10 batches)
        new_model.save('/content/gdrive/MyDrive/project deepmag/saved model 10k epoch2/')
        loss_df = pd.DataFrame({'training loss':loss_training,'validation loss':loss_val})
        loss_df.to_csv('/content/gdrive/MyDrive/project deepmag/loss_values_10k_epoch2.csv')
      print(f'{(i+1)*10} images processed')
  new_model.save('/content/gdrive/MyDrive/project deepmag/saved model 10k epoch2/')
  loss_df = pd.DataFrame({'training loss':loss_training,'validation loss':loss_val})
  loss_df.to_csv('/content/gdrive/MyDrive/project deepmag/loss_values_10k_epoch2.csv')
  print('training done on entire images in the given folder ')

In [ ]:
custom_train_loop(1000)

1/1 - 27s - loss: 1206.6229 - val_loss: 1241.2791 - 27s/epoch - 27s/step


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/project deepmag/saved model 10k epoch2/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/project deepmag/saved model 10k epoch2/assets


10 images processed
1/1 - 3s - loss: 966.8658 - val_loss: 1339.2577 - 3s/epoch - 3s/step
20 images processed
1/1 - 3s - loss: 1017.0043 - val_loss: 1140.0088 - 3s/epoch - 3s/step
30 images processed
1/1 - 3s - loss: 1040.5612 - val_loss: 1419.4796 - 3s/epoch - 3s/step
40 images processed
1/1 - 3s - loss: 1158.4275 - val_loss: 1298.2637 - 3s/epoch - 3s/step
50 images processed
1/1 - 3s - loss: 1057.2689 - val_loss: 1012.2247 - 3s/epoch - 3s/step
60 images processed
1/1 - 3s - loss: 1078.5282 - val_loss: 1107.8523 - 3s/epoch - 3s/step
70 images processed
1/1 - 3s - loss: 1158.7716 - val_loss: 1028.9080 - 3s/epoch - 3s/step
80 images processed
1/1 - 3s - loss: 1031.1503 - val_loss: 1046.5847 - 3s/epoch - 3s/step
90 images processed
1/1 - 3s - loss: 797.4772 - val_loss: 1254.0370 - 3s/epoch - 3s/step
100 images processed
1/1 - 3s - loss: 1255.0254 - val_loss: 1320.7103 - 3s/epoch - 3s/step


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/project deepmag/saved model 10k epoch2/assets


...

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/project deepmag/saved model 10k epoch2/assets


9910 images processed
1/1 - 3s - loss: 930.1639 - val_loss: 759.8265 - 3s/epoch - 3s/step
9920 images processed
1/1 - 3s - loss: 813.7997 - val_loss: 907.5522 - 3s/epoch - 3s/step
9930 images processed
1/1 - 3s - loss: 876.5565 - val_loss: 975.9142 - 3s/epoch - 3s/step
9940 images processed
1/1 - 3s - loss: 703.4850 - val_loss: 742.4993 - 3s/epoch - 3s/step
9950 images processed
1/1 - 3s - loss: 820.1434 - val_loss: 937.5692 - 3s/epoch - 3s/step
9960 images processed
1/1 - 3s - loss: 678.5617 - val_loss: 770.7531 - 3s/epoch - 3s/step
9970 images processed
1/1 - 3s - loss: 664.1614 - val_loss: 738.7281 - 3s/epoch - 3s/step
9980 images processed
1/1 - 3s - loss: 849.8876 - val_loss: 998.6653 - 3s/epoch - 3s/step
9990 images processed
1/1 - 3s - loss: 899.9100 - val_loss: 735.9904 - 3s/epoch - 3s/step
10000 images processed


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/project deepmag/saved model 10k epoch2/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/project deepmag/saved model 10k epoch2/assets


training done on entire images in the given folder 
